In [1]:
pip install surprise

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from docx import Document

In [2]:
df_final = pd.read_csv("../data/df_final.csv") 
df_240 = pd.read_csv("../data/df_240.csv") 
df_528 = pd.read_csv("../data/df_528.csv")
df_1056 = pd.read_csv("../data/df_1056.csv")

In [3]:
df_final.head()

,user_id,app_id,category,rating
0,1,1,Strategy,4
1,2,2,Puzzle,3
2,3,3,Business,2
3,4,4,Simulation,1
4,5,5,Books & Reference,3


## Generating top n recommandation to users based on 4 diffrent datasets(original+ 3 biased)

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import numpy as np

# Load datasets
df_final = pd.read_csv("../data/df_final.csv")
df_240 = pd.read_csv("../data/df_240.csv")
df_528 = pd.read_csv("../data/df_528.csv")
df_1056 = pd.read_csv("../data/df_1056.csv")

# Define datasets
datasets = {
    "df_final": df_final,
    "df_240": df_240,
    "df_528": df_528,
    'df_1056':df_1056
}

def train_and_recommend(train_df, test_users_df, k=50, top_k=25):
    # Create training user-item matrix
    train_matrix = train_df.pivot(index='user_id', columns='app_id', values='rating').fillna(0)
    user_ids = train_matrix.index
    app_ids = train_matrix.columns

    # SVD
    train_sparse = csr_matrix(train_matrix.values)
    U, sigma, Vt = svds(train_sparse, k=k)
    sigma = np.diag(sigma)
    pred_matrix = np.dot(np.dot(U, sigma), Vt)
    pred_df = pd.DataFrame(pred_matrix, index=user_ids, columns=app_ids)

    # Already rated map
    train_rated = train_df.groupby('user_id')['app_id'].apply(set).to_dict()

    # Filter users and items present in both train and test
    test_users_filtered = test_users_df[test_users_df['user_id'].isin(user_ids)]

    # Generate top-K recommendations
    top_recs = []
    for user in test_users_filtered['user_id'].unique():
        if user not in pred_df.index:
            continue
        user_pred = pred_df.loc[user]
        rated_apps = train_rated.get(user, set())
        recommendations = user_pred.drop(index=rated_apps, errors='ignore').sort_values(ascending=False).head(top_k)
        for app_id, score in recommendations.items():
            top_recs.append({'user_id': user, 'app_id': app_id, 'predicted_rating': score})

    return pd.DataFrame(top_recs)

def merge_with_category_on_app_id(recommendations_df, df_final):
    df_final_subset = df_final[['app_id', 'category']].drop_duplicates()
    merged_df = pd.merge(recommendations_df, df_final_subset, on='app_id', how='left')
    return merged_df

# Run SVD on each dataset, always test on df_final users
for name, train_df in datasets.items():
    print(f"Training on {name}, generating top-25 recommendations for df_final users...")
    recs = train_and_recommend(train_df, df_final, k=50, top_k=25)
    recs_with_category = merge_with_category_on_app_id(recs, df_final)

    # Save to correct filename
    filename = f"../result/rec/top25_{name}_with_category.csv"
    recs_with_category.to_csv(filename, index=False)
    print(f"Saved: {filename}")

Training on df_1056, generating top-25 recommendations for df_final users...


/usr/local/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/reshape.py:134: PerformanceWarning: The following operation may generate 7132971891 cells in the resulting pandas object.
  warnings.warn(


Saved: ../result/rec/top25_df_1056_with_category.csv


### Computing recommendation entropy per user, per dataset(all 6)

In [22]:
import pandas as pd
import os

# Paths
input_dir = '../result/rec'
cluster_file = '../preprocessing/df_cluster.csv'
df_cluster = pd.read_csv(cluster_file)

# Recommendation suffixes
suffixes = ['final', '240', '528', '1056']

# Load and prepare entropy data
entropy_dfs = {}
for suffix in suffixes:
    df = globals()[f'RecEnt_df_{suffix}'].rename(columns={"recommendation_entropy": f"RecEnt_{suffix}"})
    entropy_dfs[suffix] = df[['user_id', f"RecEnt_{suffix}"]]

# Merge all entropy values into one user-level dataframe
merged_user_entropy = df_cluster.copy()
for suffix in suffixes:
    merged_user_entropy = merged_user_entropy.merge(entropy_dfs[suffix], on="user_id", how="left")

# Save this clean user-level dataframe
merged_user_entropy.to_csv('../result/rec/user_level_entropy_with_cluster.csv', index=False)

# Now attach user-level info to each recommendation file
merged_recs = []

for suffix in suffixes:
    rec_file = os.path.join(input_dir, f'top25_df_{suffix}_with_category.csv')
    rec_df = pd.read_csv(rec_file)
    
    # Merge user-level info into recommendation-level file
    rec_df = rec_df.merge(merged_user_entropy, on="user_id", how="left")
    rec_df['source'] = suffix  # optional: track which recommendation version
    merged_recs.append(rec_df)

# Concatenate all into one final DataFrame
full_merged_df = pd.concat(merged_recs, ignore_index=True)

# Optional: reorder columns
cols = ['user_id', 'item_id', 'category', 'cluster', 'entropy',
        'RecEnt_final', 'RecEnt_240', 'RecEnt_528', 'RecEnt_1056', 'source']
full_merged_df = full_merged_df[[c for c in cols if c in full_merged_df.columns]]

# Save recommendation-level file (contains repetition — intended)
full_merged_df.to_csv('../result/rec/merged_entropy_with_cluster.csv', index=False)

# ✅ If needed: Deduplicated version of user-level info from rec data
user_level_df = full_merged_df[[
    'user_id', 'cluster', 'entropy',
    'RecEnt_final', 'RecEnt_240', 'RecEnt_528', 'RecEnt_1056'
]].drop_duplicates()

user_level_df.to_csv('../result/rec/merged_entropy_with_cluster.csv', index=False)

# Preview
print(user_level_df.head())


     user_id  cluster   entropy  RecEnt_final  RecEnt_240  RecEnt_528  \
0          1        2  2.809783      2.677950    2.619729    1.811208   
25         2        2  3.066907      2.553455    0.661477    0.899564   
50         3        2  3.063229      2.754332    2.243999    2.142310   
75         4        2  3.409559      2.643428    2.456143    1.873357   
100        5        2  3.427526      2.809783    2.698880    2.397922   

     RecEnt_1056  
0       0.958945  
25      0.690457  
50      1.356440  
75      0.849269  
100     1.241938  


In [19]:
RecEnt_df_1056.count()

user_id                   700111
cluster                   700111
entropy                   700111
recommendation_entropy    700111
dtype: int64

In [24]:
# Count users per cluster
cluster_counts = user_level_df['cluster'].value_counts().sort_index()

# Print nicely
print("👥 Number of users per cluster:")
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count} users")


👥 Number of users per cluster:
Cluster 0: 233371 users
Cluster 1: 233370 users
Cluster 2: 233370 users


## histogram

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Load the updated dataset
df = pd.read_csv('../result/rec/merged_entropy_with_cluster.csv')

# Output folder for base entropy visualizations
output_folder_base = '../result/rec/cluster_histograms/base'
os.makedirs(output_folder_base, exist_ok=True)

num_bins = 10

# Base entropy set
set_name = "base"
entropy_columns = ["RecEnt_final", "RecEnt_240", "RecEnt_528", "RecEnt_1056"]

for cluster_id in sorted(df["cluster"].dropna().astype(int).unique()):
    cluster_df = df[df["cluster"] == cluster_id].sort_values("entropy").reset_index(drop=True)
    users_per_bin = max(1, len(cluster_df) // num_bins)

    averages = {col: [] for col in entropy_columns}
    counts = []
    avg_entropy_per_bin = []

    for i in range(num_bins):
        start = i * users_per_bin
        end = (i + 1) * users_per_bin if i < num_bins - 1 else len(cluster_df)
        bin_df = cluster_df.iloc[start:end]
        counts.append(len(bin_df))
        avg_entropy_per_bin.append(bin_df["entropy"].mean())
        for col in entropy_columns:
            averages[col].append(bin_df[col].mean())

    # Plot
    x = np.arange(num_bins)
    width = 0.15

    plt.figure(figsize=(16, 6))
    for idx, col in enumerate(entropy_columns):
        plt.bar(x + (idx - len(entropy_columns)/2) * width, averages[col], width=width, label=col)

    plt.xlabel('User Bins (sorted by entropy)', fontsize=11)
    plt.ylabel('Average Recommendation Entropy', fontsize=11)
    plt.title(f'Cluster {cluster_id} - Entropy Across Bins ({set_name})', fontsize=13)
    plt.xticks(x, [f'Bin {i+1}\n({counts[i]})' for i in x], fontsize=10)
    plt.yticks(fontsize=10)
    plt.legend(fontsize=10)
    plt.tight_layout()

    # Save plot
    filename = os.path.join(output_folder_base, f"cluster_{cluster_id}_entropy_{set_name}.png")
    plt.savefig(filename)
    plt.close()

    # Save text file
    text_output_path = os.path.join(output_folder_base, f"cluster_{cluster_id}_entropy_{set_name}.txt")
    with open(text_output_path, 'w') as f:
        f.write(f"Cluster {cluster_id} - Average Entropy per Bin ({set_name})\n")
        f.write("Bin\tUserCount\tRawEntropy\t" + "\t".join(entropy_columns) + "\n")
        for i in range(num_bins):
            f.write(f"{i+1}\t{counts[i]}\t{avg_entropy_per_bin[i]:.4f}\t" +
                    "\t".join(f"{averages[col][i]:.4f}" for col in entropy_columns) + "\n")

print("\n✅ All base entropy plots and text files saved successfully.")


/tmp/ipykernel_1854085/1404551721.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../result/rec/merged_entropy_with_cluster.csv')



✅ All base entropy plots and text files saved successfully.


In [ ]:
import pandas as pd

# === Load merged mobile dataset ===
merged_df = pd.read_csv('../result/rec/merged_entropy_with_cluster.csv')

# === Define columns to summarize ===
rec_entropy_cols = ['rec_entropy_final', 'rec_entropy_240', 'rec_entropy_528', 'rec_entropy_1056']
summary_cols = ['entropy'] + rec_entropy_cols

# === Print average entropy per cluster ===
print("\n📊 Average Entropy Values per Cluster:\n")

for col in summary_cols:
    print(f"\n➡️ {col} (Mean per Cluster):")
    for cluster_id in sorted(merged_df['cluster'].dropna().astype(int).unique()):
        cluster_data = merged_df[merged_df['cluster'] == cluster_id]
        mean_val = cluster_data[col].mean()
        print(f"  Cluster {cluster_id}: {mean_val:.4f}")



📊 Average Entropy Values per Cluster:


➡️ entropy (Mean per Cluster):
  Cluster 0: 1.5666
  Cluster 1: 1.9818
  Cluster 2: 3.0335

➡️ rec_entropy_final (Mean per Cluster):
  Cluster 0: 2.6807
  Cluster 1: 2.6829
  Cluster 2: 2.6928

➡️ rec_entropy_240 (Mean per Cluster):
  Cluster 0: 1.8479
  Cluster 1: 2.0584
  Cluster 2: 2.2488

➡️ rec_entropy_528 (Mean per Cluster):
  Cluster 0: 0.9822
  Cluster 1: 1.1587
  Cluster 2: 1.5431

➡️ rec_entropy_1056 (Mean per Cluster):
  Cluster 0: 0.4456
  Cluster 1: 0.5911
  Cluster 2: 1.0723


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('../result/rec/merged_entropy_with_cluster.csv')

# Group by cluster and compute min and max entropy
cluster_entropy_stats = df.groupby('cluster')['entropy'].agg(['min', 'max']).reset_index()

# Print results
for _, row in cluster_entropy_stats.iterrows():
    print(f"Cluster {int(row['cluster'])} → 🔻 Min entropy: {row['min']:.4f}, 🔺 Max entropy: {row['max']:.4f}")
